In [53]:
import preprocess
import pytorch_lightning as pl
import torch
import numpy as np
from rasterio.plot import show
from torch.utils.data import DataLoader
from torch.nn import functional as F

In [54]:
class LitUNet(pl.LightningModule):
    
    def __init__(self, file_pairs, input_num=4, output_num=1, initial_feat=32, trained=False):
        super().__init__()
        self.model = model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet', in_channels=input_num, out_channels=output_num,
                                            init_features=initial_feat, pretrained=trained)
        self.file_pairs = file_pairs
        
    def forward(self, x):
        return self.model(x)
    
    def prepare_data(self):
        all_data = preprocess.GISDataset(self.file_pairs)
        # calculate the splits
        total = len(all_data)
        train = int(total*.7)
        val = int(total*.15)
        if train+(val*2) != total:
            diff = total-train-(val*2)
            train += diff
        self.train_set, self.validate_set, self.test_set = torch.utils.data.random_split(all_data, [train, val, val])
        
    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size = 64)
    
    def val_dataloader(self):
        return DataLoader(self.validate_set, batch_size=64)
    
    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=64)
    
    def configure_optimizers(self):
        optimizer=torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    
    def cross_entropy_loss(logits, labels):
        return F.nll_loss(logits, labels)

    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        
        logs = {'train_loss':loss}
        return {'loss':loss, 'log':logs}
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        return {'test_loss':loss}
    
    def test_epoch_end(self, outputs):
        avg_loss = torch.stack([x['test_loss'] for x in outputs]).mean()
        tensorboard_logs = {'test_loss':avg_loss}
        return {'avg_test_loss':avg_loss, 'log':tensorboard_logs}
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        logits = self.forward(x)
        loss = self.cross_entropy_loss(logits, y)
        return {'val_loss':loss}
    
    def validation_epoch_end(self, outputs):
        # called at the end of the validation epoch
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        tensorboard_logs = {'val_loss':avg_loss}
        return {'avg_val_loss':avg_loss, 'log':tensorboard_logs}
        

In [55]:
twelve_img = "/vol/ml/EphemeralStreamData/Ephemeral_Channels/Imagery/vhr_2012_refl.img"
twelve_shp = "/vol/ml/EphemeralStreamData/Ephemeral_Channels/Reference/reference_2012_merge.shp"

In [56]:
model = LitUNet([(twelve_img, twelve_shp)])

Using cache found in /homes/mzvyagin/.cache/torch/hub/mateuszbuda_brain-segmentation-pytorch_master


In [57]:
trainer = pl.Trainer(gpus=[2])

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [2,3]


In [ ]:
trainer.fit(model)

/homes/mzvyagin/anaconda3/envs/gis/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))
